In [1]:
import os
import sys
import uuid

import random

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset preprocessing

In [2]:
vdjdb_raw = pd.read_csv('data/raw/vdjdb_paired_010923.tsv', sep='\t')
iedb_raw = pd.read_csv('data/raw/iedb_010923.csv')
mcpas_raw = pd.read_csv('data/raw/mcpas-tcr_010923.csv', encoding='latin1')

/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_55974/1072130901.py:2: DtypeWarning: Columns (8,10,11,15,16,17,18,20,21,22,23,25,30,31,36,37,44,45,46,47,48,49,50,51,52,59,60,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  iedb_raw = pd.read_csv('data/raw/iedb_010923.csv')
/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_55974/1072130901.py:3: DtypeWarning: Columns (13,17,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  mcpas_raw = pd.read_csv('data/raw/mcpas-tcr_010923.csv', encoding='latin1')


## VDJdb

In [3]:
vdj_cols = ['complex.id', 'CDR3', 'V', 'J', 'Species', 'MHC A','MHC class', 'Epitope']
print('VDJdb raw:', vdjdb_raw.shape)

vdjdb = vdjdb_raw.dropna()
vdjdb_a = vdjdb[vdjdb['Gene']=='TRA'][vdj_cols].copy()
vdjdb_b = vdjdb[vdjdb['Gene']=='TRB'][vdj_cols].copy()

vdjdb = pd.merge(vdjdb_a, vdjdb_b, on='complex.id')
print('VDJdb paired:', vdjdb.shape)

assert (vdjdb['Species_x'] == vdjdb['Species_y']).all()
assert (vdjdb['Epitope_x'] == vdjdb['Epitope_y']).all()
assert (vdjdb['MHC A_x'] == vdjdb['MHC A_y']).all()

vdj_cols = ['complex.id', 'CDR3_x', 'V_x', 'J_x', 'CDR3_y', 'V_y', 'J_y', 'Species_y', 'MHC A_y', 'MHC class_y', 'Epitope_y']
vdjdb = vdjdb[vdj_cols].copy()
vdj_cols = ['complex.id', 'cdr3a', 'va', 'ja', 'cdr3b', 'vb', 'jb', 'species', 'mhc', 'mhc_class', 'epitope']
vdjdb.columns = vdj_cols

# final data formating
try:
    vdjdb = vdjdb.drop('complex.id', axis=1)
except:
    pass
vdjdb = vdjdb.replace('MHCI', 1) # name formatting
vdjdb = vdjdb.replace('HomoSapiens', 'human') # name formatting
vdjdb = vdjdb[vdjdb['species']=='human'].copy() # keep only humans
vdjdb = vdjdb.drop_duplicates()
print("preprocessed vdjdb:", vdjdb.shape)
vdjdb.head()


VDJdb raw: (3050, 17)
VDJdb paired: (1397, 15)
preprocessed vdjdb: (1102, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
0,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEKGGL
1,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEKGGL
2,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,human,HLA-B*08,1,FLKEKGGL
3,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEQGGL
4,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEQGGL


## IEDB

IEDB queried for MHC 1 and human only

In [4]:
subset = ['Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated']
iedb = iedb_raw.dropna(subset=subset)
iedb = iedb.astype('str')
print("Full IEDB:", iedb.shape)

iedb_base_cols = ['Group Receptor ID', 'Receptor ID', 'Description', 'Organism', 'MHC Allele Names']

iedb_cols = [col for col in iedb.columns if "Curated" in col and ("CDR3" in col or "V" in col or "J" in col)]
iedb_cols = ['Description', 'Organism', 'MHC Allele Names', 'Curated Chain 1 V Gene', 'Curated Chain 1 J Gene', 'Chain 1 CDR3 Curated',  
            'Curated Chain 2 V Gene','Curated Chain 2 J Gene', 'Chain 2 CDR3 Curated']
iedb = iedb[iedb_cols]
print("IEDB Selected Columns:", iedb.shape)

iedb = iedb.replace('nan', np.nan)
iedb = iedb.dropna()
iedb.columns = ['epitope', 'species', 'mhc', 'va', 'ja', 'cdr3a', 'vb', 'jb', 'cdr3b']
iedb['mhc_class'] = 1
iedb['species'] = 'human' # IEDB filtered for only human

iedb = iedb[vdjdb.columns]
print('IEDB remove nan:', iedb.shape)
iedb = iedb.drop_duplicates()
print('IEDB remove duplicates:', iedb.shape)

# we need the CDR3 ends for 3D generation, one way to get this is to check if the sequence starts with C
iedb = iedb[iedb['cdr3a'].str.startswith('C')]
iedb = iedb[iedb['cdr3b'].str.startswith('C')]
print('IEDB with cdr3 ends only:', iedb.shape)

# remove rows with sequences containing #
print('IEDB cdr3a sequences with #:', iedb['cdr3a'].str.contains('#').sum())
iedb = iedb[iedb['cdr3a'].str.contains('#')==False]
print('IEDB cdr3b sequences with #:', iedb['cdr3b'].str.contains('#').sum())
iedb = iedb[iedb['cdr3b'].str.contains('#')==False]



# VJ name formating for cases where V, J gene starts with 'TCR' instead of 'TR'
iedb = iedb.replace('TCR', 'TR', regex=True)

print('IEDB final:', iedb.shape)

iedb

Full IEDB: (24032, 71)
IEDB Selected Columns: (24032, 9)
IEDB remove nan: (6093, 10)
IEDB remove duplicates: (6002, 10)
IEDB with cdr3 ends only: (5483, 10)
IEDB cdr3a sequences with #: 22
IEDB cdr3b sequences with #: 0
IEDB final: (5461, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
81,CAVRPTSGGSYIPTF,TRAV21*01,TRAJ6*01,CASSYVGNTGELFF,TRBV6-5*01,TRBJ2-2*01,human,HLA-A*02:01,1,SLLMWITQC
173,CAGGTGNQFYF,TRAV35*02,TRAJ49*01,CAISEVGVGQPQHF,TRBV10-3,TRBJ1-5*01,human,HLA-A*02:01,1,AAGIGILTV
617,CALSEAGTGGSYIPTF,TRAV19,TRAJ6,CASSMFVGQPQHF,TRBV19,TRBJ1-5,human,HLA-A*02:01,1,GILGFVFTL
618,CAVSVEETSGSRLTF,TRAV41,TRAJ58,CASSFFHNNEQFF,TRBV19,TRBJ2-1,human,HLA-A*02:01,1,GILGFVFTL
619,CAYRSARDSSYKLIF,TRAV38-2/DV8*01,TRAJ12*01,CASSDHSVTGISSPLHF,TRBV7-9*03,TRBJ1-6*02,human,HLA-B7,1,TPRVTGGGAM
...,...,...,...,...,...,...,...,...,...,...
24299,CGAGETSGSRLTF,TRAV21,TRAJ58,CSVNLGGPTDTQYF,TRBV29-1,TRBJ2-3,human,HLA-B*07:02,1,KPVETSNSF
24300,CALEGSQGNLIF,TRAV9-2,TRAJ42,CSVPDGAEPYGYTF,TRBV20-1,TRBJ1-2,human,HLA-A*01:01,1,TTDPSFLGRY
24301,CLVGNTGGFKTIF,TRAV4,TRAJ9,CSVPDRGNTEAFF,TRBV29-1,TRBJ1-1,human,HLA-B*07:02,1,SPRWYFYYL
24303,CAPSRHAGNNRKLIW,TRAV9-2,TRAJ38,CSVQGGTNEKLFF,TRBV29-1,TRBJ1-4,human,HLA-A*01:01,1,VSDGGPNLY


In [5]:
iedb[iedb['cdr3a'].str.contains('#')==True].shape


(0, 10)

## McPAS-TCR

In [6]:
mcpas_cols = ['CDR3.alpha.aa', 'CDR3.beta.aa', 'Species', 'Epitope.peptide', 'MHC', 'TRAV', 'TRAJ', 'TRBV', 'TRBJ']
mcpas = mcpas_raw[mcpas_cols].copy()
print('McPAS-TCR raw:', mcpas.shape)
mcpas = mcpas.dropna()
print('McPAS-TCR drop na:', mcpas.shape)
mcpas = mcpas.drop_duplicates()
print('McPAS-TCR drop duplicates:', mcpas.shape)

# formatting
mcpas['Species'] = mcpas['Species'].str.lower()
mcpas.columns = ['cdr3a', 'cdr3b', 'species', 'epitope', 'mhc', 'va', 'ja', 'vb', 'jb']
mcpas['mhc_class'] = 1
mcpas = mcpas[vdjdb.columns].copy()

# remove rows with sequences containing #
print('McPAS cdr3a sequences with #:', mcpas['cdr3a'].str.contains('#').sum())
mcpas = mcpas[mcpas['cdr3a'].str.contains('#')==False]
print('McPAS cdr3b sequences with #:', mcpas['cdr3b'].str.contains('#').sum())
mcpas = mcpas[mcpas['cdr3b'].str.contains('#')==False]

print('McPAS-TCR final:', mcpas.shape)

mcpas 

McPAS-TCR raw: (39985, 9)
McPAS-TCR drop na: (3124, 9)
McPAS-TCR drop duplicates: (2914, 9)
McPAS cdr3a sequences with #: 0
McPAS cdr3b sequences with #: 0
McPAS-TCR final: (2914, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
73,CALGLMSNYNVLYF,TRAV4,TRAJ14-4,CASSSGLGGTLYF,TRBV10,TRBJ2-4,mouse,H-2Db,1,SSGVENPGGYCLTKW
80,CAAETTASLGKLQF,TRAV11,TRAJ9,CASGDHGLSYEQYF,TRBV13-3,TRBJ2-6,mouse,H-2b,1,DEPLTSLTPRCNTAWNRLKL
84,CALGDRGSGGSNYK,TRAV4,TRAJ3DT,CAWSRTGGNSDYTF,TRBV31,TRBJ1-2,mouse,H-2b,1,DEPLTSLTPRCNTAWNRLKL
120,CAAEASSSFSKLVF,TRAV11-1,TRAJ42,CASAPDRGGERLF,TRBV8-2,TRBJ1-4,mouse,H-2q,1,GPEGAQGPRGEPGTP
121,CAAEASSSFSKLVF,TRAV11-1,TRAJ42,CASAPDRGGERLF,TRBV13-2,TRBJ1-4,mouse,H-2q,1,GPEGAQGPRGEPGTP
...,...,...,...,...,...,...,...,...,...,...
39028,CALDGPSNTGKLIF,TRAV16,TRAJ37,CATSESSGQTYEQYF,TRBV15,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39029,CATDAEGNNRLAF,TRAV17,TRAJ7,CASSIFGGGLGEQFF,TRBV19,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL
39030,CGAVGYQKVTF,TRAV34,TRAJ13,CALNGEISYNEQFF,TRBV2,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39031,CAVIWYNNNDMRF,TRAV8-1,TRAJ43,CASSQGVNTGELFF,TRBV4-2,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL


## Combine

In [7]:
positives = pd.concat((iedb, vdjdb, mcpas), axis=0)
print("concate db:", positives.shape)
positives = positives.drop_duplicates(subset=['cdr3a', 'cdr3b', 'epitope'], keep='last', ignore_index=True)
print("concate db remove duplicates:", positives.shape)
positives['uuid'] = [uuid.uuid4() for _ in range(len(positives.index))]

positives

concate db: (9477, 10)
concate db remove duplicates: (8411, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope,uuid
0,CAVSVEETSGSRLTF,TRAV41,TRAJ58,CASSFFHNNEQFF,TRBV19,TRBJ2-1,human,HLA-A*02:01,1,GILGFVFTL,30ec9899-b58a-45cc-955b-0a58427d4262
1,CAVNKGYGQNFVF,TRAV12-2*02,TRAJ26*01,CASSPAGISYNSPLHF,TRBV7-9*03,TRBJ1-6*01,human,HLA-B8,1,ELRRKMMYM,9e5581b5-21e3-49ab-be94-c5fde3fad1ed
2,CAVRDSSYSGAGSYQLTF,TRAV3*01,TRAJ28*01,CASSRLAGASTDTQYF,TRBV7-3*01,TRBJ2-3*01,human,HLA-B8,1,QIKVRVDMV,dddcde4f-2ac3-47a5-9b57-59a5c248fdb3
3,CAVSDYGQNFVF,TRAV21*01,TRAJ26*01,CASSRLSSNTDTQYF,TRBV7-3*01,TRBJ2-3*01,human,HLA-B8,1,QIKVRVDMV,ef516c00-b682-400d-b738-3e1d4b785303
4,CATAQVYSGGGADGLTF,TRAV17*01,TRAJ45*01,CASSRLAGNTDTQYF,TRBV7-3*01,TRBJ2-3*01,human,HLA-B8,1,QIKVRVDMV,7067f110-96a0-42b9-8ffc-58bed38ca60a
...,...,...,...,...,...,...,...,...,...,...,...
8406,CALDGPSNTGKLIF,TRAV16,TRAJ37,CATSESSGQTYEQYF,TRBV15,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL,929578e2-0c45-44d1-b09d-ead06991941c
8407,CATDAEGNNRLAF,TRAV17,TRAJ7,CASSIFGGGLGEQFF,TRBV19,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL,fc74b391-4efe-449f-9e9b-c2169e072eb8
8408,CGAVGYQKVTF,TRAV34,TRAJ13,CALNGEISYNEQFF,TRBV2,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL,41b94edc-bb15-47ad-87e1-86abb06f8a0f
8409,CAVIWYNNNDMRF,TRAV8-1,TRAJ43,CASSQGVNTGELFF,TRBV4-2,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL,19c3188a-5c94-4a74-a22f-3ab994374b46


In [8]:
complex = []
for i in range(len(positives)):
    complex.append((positives['cdr3a'].iloc[i], positives['cdr3b'].iloc[i], positives['epitope'].iloc[i]))
positives['complex'] = complex
positives_dict = positives.drop_duplicates(subset='complex').set_index('complex', drop=True).to_dict(orient='index')
try:
    positives = positives.drop('complex', axis=1)
except:
    pass
print('unique tcr-peptide pairs:', len(positives_dict.keys()))

unique tcr-peptide pairs: 8411


# Negative sample generation

In [9]:
# create a lookup dictionary of all positive binding samples
def lookup_dict(df: pd.DataFrame, cdr3a: str, cdr3b: str, epitope: str) -> dict:
    complex = []
    for i in range(len(df.index)):
        complex.append((df[cdr3a].iloc[i], df[cdr3b].iloc[i], df[epitope].iloc[i]))
    df['complex'] = complex
    df_dict = df.drop_duplicates(subset='complex').set_index('complex', drop=True).to_dict(orient='index')
    return df_dict

# vdjdb
vdjdb_lc_raw = pd.read_csv('data/raw/vdjdb_low_confidence.tsv', sep='\t')
vdjdb_lc_a = vdjdb_lc_raw[vdjdb_lc_raw['Gene']=='TRA'].copy()
vdjdb_lc_b = vdjdb_lc_raw[vdjdb_lc_raw['Gene']=='TRB'].copy()
vdjdb_lc = pd.merge(vdjdb_lc_a, vdjdb_lc_b, on='complex.id')

vdjdb_lc_dict = lookup_dict(vdjdb_lc, 'CDR3_x', 'CDR3_y', 'Epitope_y')

#iedb
subset = ['Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated']
iedb_lc = iedb_raw.dropna(subset=subset)
iedb_lc = iedb_lc.astype('str')

iedb_lc_dict = lookup_dict(iedb_lc, 'Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated', 'Description')

#mcpas-tcr
mcpas_lc = mcpas_raw.dropna(subset=['CDR3.alpha.aa', 'CDR3.beta.aa', 'Epitope.peptide'])

mcpas_lc_dict = lookup_dict(mcpas_lc, 'CDR3.alpha.aa', 'CDR3.beta.aa', 'Epitope.peptide')


/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_55974/1627262767.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['complex'] = complex


In [10]:
print(vdjdb_lc_raw['CDR3'].str.contains('#').sum())
print(iedb_lc['Chain 1 CDR3 Curated'].str.contains('#').sum())
print(mcpas_lc['CDR3.alpha.aa'].str.contains('#').sum())

0
28
1


In [11]:
i = 0
negatives = []
while i<len(positives.index):
    sample_idx = random.choice([k for k in range(len(positives)) if k!=i])
    neg_complex = (positives['cdr3a'].iloc[sample_idx], positives['cdr3b'].iloc[sample_idx], positives['epitope'].iloc[i])
    try:
        # sample already exists in either positives/vdjdb/iedb/mcpas-tcr and it's positive --> we do not want this
        positives_dict[neg_complex]
    except KeyError:
        try:
            vdjdb_lc_dict[neg_complex]
        except KeyError:
            try:
                iedb_lc_dict[neg_complex]
            except KeyError:
                try:
                    mcpas_lc_dict[neg_complex]
                except KeyError:
                    # negative sample is not positive
                    sample = positives.iloc[sample_idx].copy()
                    sample['epitope'] =  positives['epitope'].iloc[i]
                    sample['mhc'] = positives['mhc'].iloc[i]
                    if tuple(sample.values) not in negatives: # check if generated negative already exists
                        negatives.append(tuple(sample.values))
                        i+=1
                        
negatives = pd.DataFrame(negatives, columns=positives.columns)
print('generated negatives:', negatives.shape)
negatives = negatives.drop_duplicates() # this confirms there are no duplicates
print('generated negatives drop duplicates:', negatives.shape)

negatives['uuid'] = [uuid.uuid4() for _ in range(len(negatives.index))]
print('negative samples:', negatives.shape)
negatives

generated negatives: (8411, 11)
generated negatives drop duplicates: (8411, 11)
negative samples: (8411, 11)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope,uuid
0,CVVSDWAGNMLTF,TRAV8-2,TRAJ39,CASSERLDLAGGRNEQFF,TRBV6-1,TRBJ2-1,human,HLA-A*02:01,1,GILGFVFTL,a0d44c8c-dc19-481c-b7ab-5f064ec6d4d1
1,CALGAGSNYKLTF,TRAV6D-6:02,TRAJ5-01,CASSLGGEVFF,TRBV2-01,TRBJ1-1:01,mouse,HLA-B8,1,ELRRKMMYM,c6897bea-8678-4196-81e5-7036dd370b2a
2,CIPNSNSGYALNF,TRAV26-2,TRAJ41,CASTPGVYTDTQYF,TRBV27,TRBJ2-3,human,HLA-B8,1,QIKVRVDMV,d6c7c2c3-5fd7-4038-b680-18f93fb67150
3,CALVPSNTNKVVF,TRAV6D-4:01,TRAJ3-01,CASSLGGYEQYF,TRBV2-01,TRBJ2-7:01,mouse,HLA-B8,1,QIKVRVDMV,3f6891c6-3507-4794-b81a-e34d54d937d7
4,CAVNGNNNDMRF,TRAV12-2*01,TRAJ43*01,CASAWGGMTNEQYF,TRBV19*01,TRBJ2-7*01,human,HLA-B8,1,QIKVRVDMV,ecca3c08-aea0-4fb4-b094-0bf331602732
...,...,...,...,...,...,...,...,...,...,...,...
8406,CAGGGETSGSRLTF,TRAV27,TRAJ58,CASSYSLWGIGDTQYF,TRBV6-6,TRBJ2-3,human,HLA-A2:01,1,FLCMKALLL,7074633d-7d68-4c46-a085-61b88696307e
8407,CATKGNVLHC,TRAV17,TRAJ35,CASSDRAGNTIYF,TRBV19,TRBJ1-3,human,HLA-A2:01,1,FLCMKALLL,833975c1-c6d3-495a-b340-828a6e551145
8408,CAVNSDYKLSF,TRAV8-1,TRAJ20,CASALAGGPAYEQYF,TRBV5-6,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL,87b49edd-ec18-4b19-8a9f-4958e89d09e0
8409,CAVGDGGATNKLIF,TRAV2,TRAJ32,CASSYWGGREDTQYF,TRBV6-5,TRBJ2-3,human,HLA-A2:01,1,FLCMKALLL,57a6f794-d226-4396-b9e5-176599443d59


In [12]:
# check if any overlap between negatives and positives
pd.merge(positives, negatives, on=['cdr3a', 'cdr3b', 'epitope'])

,cdr3a,va_x,ja_x,cdr3b,vb_x,jb_x,species_x,mhc_x,mhc_class_x,epitope,uuid_x,va_y,ja_y,vb_y,jb_y,species_y,mhc_y,mhc_class_y,uuid_y


# Saving Data

In [14]:
save_path = 'data/preprocessed'
positives.to_csv(os.path.join(save_path, 'positives.tsv'), sep='\t')
negatives.to_csv(os.path.join(save_path, 'negatives.tsv'), sep='\t')